In [72]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

In [73]:
URL_BASES = "../../Databases/MIMIC/"
patient = "patients.csv"
admission = "admissions.csv"
diagnostics = "diagnoses_icd.csv"
icd_diagnostics = "d_icd_diagnoses.csv"
lab = "labevents.csv"
lab_items = "d_labitems.csv"
chart_items = "d_items.csv"
chart = "chartevents.csv"

url_patient = URL_BASES + patient
url_admission = URL_BASES + admission
url_diagnostics = URL_BASES + diagnostics
url_icd_diagnostics = URL_BASES + icd_diagnostics
url_lab = URL_BASES + lab
url_lab_items = URL_BASES + lab_items
url_chart_items = URL_BASES + chart_items
url_chart = URL_BASES + chart

In [74]:
def aggregate_events(df):
    Albumine = df.loc[df['itemid'].isin([52022, 53138, 50862, 53085]), 'value']
    Lactate = df.loc[df['itemid'].isin([50813, 52442, 53154]), 'value']
    Bicarbonate = df.loc[df['itemid'].isin([50813, 52442, 53154]), 'value']
    Chloride = df.loc[df['itemid'].isin( [50902, 52535]), 'value']
    Hemoglobin = df.loc[df['itemid'].isin([50811, 51222, 51640]), 'value']
    Sodium = df.loc[df['itemid'].isin([50983, 52623]), 'value']
    Glucose = df.loc[df['itemid'].isin([50809, 50931, 52569]), 'value']
    Troponin = df.loc[df['itemid'].isin([51002, 51003, 52642]), 'value']
    Platelets = df.loc[df['itemid'].isin([51240]), 'value']
    INR = df.loc[df['itemid'].isin([51237, 51675]), 'value']
    Creatinine = df.loc[df['itemid'].isin([50813, 52442, 53154]), 'value']
    Bun = df.loc[df['itemid'].isin([51842]), 'value']
    Bilirubin = df.loc[df['itemid'].isin([50885, 53089]), 'value']
    AST = df.loc[df['itemid'].isin([50878, 53088]), 'value']
    ALT = df.loc[df['itemid'].isin([50861, 53084]), 'value']
    C_reactive = df.loc[df['itemid'].isin([50889]), 'value']
    ESR = df.loc[df['itemid'].isin([51288]), 'value']
    Bands = df.loc[df['itemid'].isin([51144]), 'value']
    WBC = df.loc[df['itemid'].isin([51301]), 'value']
    Oxygen_saturation = df.loc[df['itemid'].isin([50817]), 'value']
    Temperature = df.loc[df['itemid'].isin([223762]), 'value']
    Systolic_pressure = df.loc[df['itemid'].isin([227242, 227243]), 'value']
        
    series = {
        'Albumine': Albumine.values[0] if len(Albumine) > 0 else None,
        'Lactate': Lactate.values[0] if len(Lactate) > 0 else None,
        'Bicarbonate': Bicarbonate.values[0] if len(Bicarbonate) > 0 else None,
        'Chloride': Chloride.values[0] if len(Chloride) > 0 else None,
        'Hemoglobin': Hemoglobin.values[0] if len(Hemoglobin) > 0 else None,
        'Sodium': Sodium.values[0] if len(Sodium) > 0 else None,
        'Glucose': Glucose.values[0] if len(Glucose) > 0 else None,
        'Troponin': Troponin.values[0] if len(Troponin) > 0 else None,
        'Platelets': Platelets.values[0] if len(Platelets) > 0 else None,
        'INR': INR.values[0] if len(INR) > 0 else None,
        'Creatinine': Creatinine.values[0] if len(Creatinine) > 0 else None,
        'Bun': Bun.values[0] if len(Bun) > 0 else None,
        'Bilirubin': Bilirubin.values[0] if len(Bilirubin) > 0 else None,
        'AST': AST.values[0] if len(AST) > 0 else None,
        'ALT': ALT.values[0] if len(ALT) > 0 else None,
        'C_reactive': C_reactive.values[0] if len(C_reactive) > 0 else None,
        'ESR': ESR.values[0] if len(ESR) > 0 else None,
        'Bands': Bands.values[0] if len(Bands) > 0 else None,
        'WBC': WBC.values[0] if len(WBC) > 0 else None,
        'Oxygen_saturation': Oxygen_saturation.values[0] if len(Oxygen_saturation) > 0 else None,
        'Temperature': Temperature.values[0] if len(Temperature) > 0 else None,
        'Systolic_pressure': Systolic_pressure.values[0] if len(Systolic_pressure) > 0 else None
    }
        
    return pd.Series(series)

In [75]:
itemids = [50817,51301,51144,51288,50889,50861,53084,50878,53088,50885,53089,51842,52024,51237,51675,51240,51002,51003,52642,50809,50931,52569,50983,52623,50811,51222,51640,50902,52535,50882,50813,52442,53154,52022,53138,50862,53085]

In [76]:
df_labevents = pd.read_csv(url_lab)
df_selected_labevents = df_labevents.loc[:, ['subject_id', 'charttime','itemid', 'value']]
df_selected_labevents = df_selected_labevents[df_selected_labevents['itemid'].isin(itemids)]
df_selected_labevents = df_selected_labevents.groupby(['subject_id', 'charttime']).apply(aggregate_events).reset_index()
df_selected_labevents[(df_selected_labevents["subject_id"].isin([10035631]))].sort_values(by='charttime')   
display(df_selected_labevents[df_selected_labevents["subject_id"].isin([10035631])].sort_values(by='charttime'))

/tmp/ipykernel_228085/610133703.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_selected_labevents = df_selected_labevents.groupby(['subject_id', 'charttime']).apply(aggregate_events).reset_index()


,subject_id,charttime,Albumine,Lactate,Bicarbonate,Chloride,Hemoglobin,Sodium,Glucose,Troponin,...,Bilirubin,AST,ALT,C_reactive,ESR,Bands,WBC,Oxygen_saturation,Temperature,Systolic_pressure
3566,10035631,2112-09-17 20:30:00,None,None,None,102,9.7,139,___,None,...,1.3,48,41,None,None,0,___,None,NaN,NaN
3567,10035631,2112-09-18 03:15:00,None,None,None,None,9.5,None,None,None,...,None,None,None,None,None,___,___,None,NaN,NaN
3568,10035631,2112-09-19 06:15:00,None,None,None,110,8.7,144,___,None,...,0.9,32,28,None,None,6,___,None,NaN,NaN
3569,10035631,2112-09-19 19:30:00,None,None,None,None,7.2,None,None,None,...,None,None,None,None,None,None,___,None,NaN,NaN
3570,10035631,2112-09-20 04:23:00,None,None,None,108,8.4,139,___,None,...,0.9,34,27,None,None,0,___,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3837,10035631,2116-03-08 22:55:00,None,None,None,96,None,135,___,None,...,None,None,None,None,None,None,None,None,NaN,NaN
3838,10035631,2116-03-08 23:18:00,None,1.2,1.2,None,None,None,None,None,...,None,None,None,None,None,None,None,None,NaN,NaN
3839,10035631,2116-03-09 05:31:00,2.4,None,None,98,9.8,133,___,None,...,3.6,72,47,None,None,None,0.1,None,NaN,NaN
3840,10035631,2116-03-09 05:43:00,None,1.2,1.2,None,None,None,None,None,...,None,None,None,None,None,None,None,None,NaN,NaN
